In [ ]:
import scipy.io as scio
import numpy as np    
import matplotlib.pyplot as plt
import sys
import os
import math
import pprint
import cv2
from scipy.misc import imsave
from helper import *
import re


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [ ]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

In [ ]:
def list_all_files(directory):
    from os import listdir
    from os.path import isfile, join
    all_labels_files = [f for f in listdir(directory) if isfile(join(directory, f))]
    all_labels_files.sort()
    return all_labels_files

In [ ]:
new_file_list = ['H1_N01848_LA_1_272', 'H1_N01848_LA_1_388', 'H1_N01848_LV_1_194', 'H1_N01848_LV_1_300', 'H1_N01848_RVS_1_244', 'H1_N01848_RVS_1_300', 'H2_N02047_LA_1_241', 'H2_N02047_LA_1_257', 'H2_N02047_LVS_1_400', 'H2_N02047_LVS_1_420', 'H2_N02047_LV_1_194', 'H2_N02047_LV_1_400', 'H2_N02047_RA_1_380', 'H2_N02047_RA_1_400', 'H2_N02047_RVS_1_334', 'H2_N02047_RVS_1_400', 'H3_N02186_LA_1_400', 'H3_N02186_LA_1_416', 'H3_N02186_LVS_1_265', 'H3_N02186_LVS_1_676', 'H3_N02186_RA_1_380', 'H3_N02186_RA_1_400', 'H3_N02186_RVS_1_400', 'H3_N02186_RVS_1_420', 'H3_N02186_RV_1_270', 'H3_N02186_RV_1_400', 'H4_N03210_LVS_1_390', 'H4_N03210_LVS_1_400', 'H4_N03210_LV_1_400', 'H4_N03210_RA_1_400', 'H4_N03210_RA_1_440', 'H4_N03210_RVS_1_400', 'H4_N03210_RVS_1_420', 'H4_N03210_RV_1_390', 'H4_N03210_RV_1_400', 'H5_N03290_LA_1_380', 'H5_N03290_LA_1_400', 'H5_N03290_LVS_1_361', 'H5_N03290_LVS_1_400', 'H5_N03290_LV_1_400', 'H5_N03290_LV_1_420', 'H5_N03290_RVS_1_185', 'H5_N03290_RVS_1_400', 'H6_N03320_LA_1_391', 'H6_N03320_LA_1_450', 'H6_N03320_LVS_1_400', 'H6_N03320_LVS_1_455', 'H6_N03320_LV_1_404', 'H6_N03320_LV_1_620', 'H6_N03320_RA_1_420', 'H6_N03320_RVS_1_400', 'H6_N03320_RVS_1_476', 'H7_N03555_LVS_1_306', 'H7_N03555_LV_1_240', 'H7_N03555_RA_1_302', 'H7_N03555_RVS_1_41', 'H7_N03555_RV_1_221', 'H8_N03585_RA_1_345', 'H8_N03585_RVS_1_398', 'H9_N03857_LVS_1_420', 'H9_N03857_LV_1_580', 'H9_N03857_RVS_1_400', 'H9_N03857_RVS_1_420', 'H9_N03857_RV_1_380', 'H9_N03857_RV_1_425']

In [ ]:
# Removing Unwanted Files
atrium_files = []
ventricle_files = []
pruned_files = ['con_H1_N01848_LV_1_194.tif','con_H1_N01848_LV_1_300.tif','con_H2_N02047_LVS_1_400.tif','con_H2_N02047_LVS_1_420.tif','con_H3_N02186_LVS_1_265.tif','con_H3_N02186_RV_1_270.tif','con_H3_N02186_RV_1_400.tif','con_H4_N03210_LV_1_400.tif','con_H4_N03210_RV_1_390.tif','con_H4_N03210_RV_1_400.tif','con_H5_N03290_LV_1_420.tif','con_H6_N03320_LVS_1_455.tif','con_H6_N03320_LV_1_404.tif','con_H6_N03320_RA_1_420.tif','con_H8_N03585_RA_1_345.tif']
pruned_files = [x.replace('con_','') for x in pruned_files]

# for count, i in enumerate(new_file_list):
#     print(count,i)
# print(pruned_files)
for files in new_file_list:
    if files[11] == 'V':
        ventricle_files.append(files)
    elif files[11] == 'A':
        atrium_files.append(files)
# print(atrium_files)

In [ ]:
def remove_files_atrium(mydir, ext):
    filelist = [ f for f in os.listdir(mydir) if f.endswith(ext) ]
    
#     print(len(filelist))
    for f in filelist:
        if f[:4] == 'con_':
            new_f = f[4:]
        elif f[:6] == 'label_':
            new_f = f[6:]
        elif f[:7] == 'weight_':
            new_f = f[7:]
        elif f[:3] == 'id_':
            new_f = f[3:]
        else:
            new_f = f
        if new_f[11] == 'V':
            os.remove(os.path.join(mydir, f))

def remove_files_ventricle(mydir, ext):
    filelist = [ f for f in os.listdir(mydir) if f.endswith(ext) ]
#     print(len(filelist))
    for f in filelist:
        if f[:4] == 'con_':
            new_f = f[4:]
        elif f[:6] == 'label_':
            new_f = f[6:]
        elif f[:7] == 'weight_':
            new_f = f[7:]
        elif f[:3] == 'id_':
            new_f = f[3:]
        else:
            new_f = f

        if new_f[11] == 'A':
            os.remove(os.path.join(mydir, f))

def remove_files_pruned(mydir, ext):
    filelist = [ f for f in os.listdir(mydir) if f.endswith(ext) ]
    hated_files = []
    for f in filelist:
        if f[:4] == 'con_':
            new_f = f[4:]
        elif f[:6] == 'label_':
            new_f = f[6:]
        elif f[:7] == 'weight_':
            new_f = f[7:]
        elif f[:3] == 'id_':
            new_f = f[3:]
        else:
            new_f = f
        for x in pruned_files:
             if x[:19].replace('.','_') == new_f[:19].replace('.','_'):
                os.remove(os.path.join(mydir, f))

### Remove Files from Pruned

In [ ]:
# Setting the directories
import os
cwd = os.getcwd() + '/datasets-24-aug/OCTData/pruned/'

print(cwd)
directories = sorted(os.listdir(path=cwd))
directories = directories[1:]

# Standard folder
folders = {}
for i in directories:
    name = i+'_folder'
    folders[name] = cwd + i+'/'
print('Name: {}\nDirectory: {}\n'.format('whole_raw_image_folder', folders['whole_raw_image_folder']))

# Train folder
train_folders = {}
for i in directories:
    name = i
    train_folders[name] = cwd + i+'/Train/'

# Transpose folder
transpose_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/transpose/'):
        transpose_folders[name] = cwd + i+'/transpose/'

# Train/weight folder
weight_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/weights/'):
        weight_folders[name] = cwd + i+'/Train/weights/'
    
# Train/segmented_ids folder
id_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/segmented_ids/'):
        id_folders[name] = cwd + i+'/Train/segmented_ids/'

In [ ]:
# Standard folder - .tif
for folder in folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_pruned(folders[folder],ext)

# Train folder - .tif
# train_folders = {}
for folder in train_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_pruned(train_folders[folder],ext)

# Transpose folder
# transpose_folders = {}
for folder in transpose_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_pruned(transpose_folders[folder],ext)

# Train/weight folder
# weight_folders = {}
for folder in weight_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_pruned(weight_folders[folder],ext)

# Train/segmented_ids folder
# id_folders = {}
for folder in id_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_pruned(id_folders[folder],ext)

### Remove files from Atrium

In [ ]:
# Setting the directories
import os
cwd = os.getcwd() + '/datasets-24-aug/OCTData/Atrium/'

print(cwd)
directories = sorted(os.listdir(path=cwd))
directories = directories[1:]

# Standard folder
folders = {}
for i in directories:
    name = i+'_folder'
    folders[name] = cwd + i+'/'
print('Name: {}\nDirectory: {}\n'.format('whole_raw_image_folder', folders['whole_raw_image_folder']))

# Train folder
train_folders = {}
for i in directories:
    name = i
    train_folders[name] = cwd + i+'/Train/'

# Transpose folder
transpose_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/transpose/'):
        transpose_folders[name] = cwd + i+'/transpose/'

# Train/weight folder
weight_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/weights/'):
        weight_folders[name] = cwd + i+'/Train/weights/'
    
# Train/segmented_ids folder
id_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/segmented_ids/'):
        id_folders[name] = cwd + i+'/Train/segmented_ids/'
    

In [ ]:
# Standard folder - .tif
for folder in folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_atrium(folders[folder],ext)

# Train folder - .tif
# train_folders = {}
for folder in train_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_atrium(train_folders[folder],ext)

# Transpose folder
# transpose_folders = {}
for folder in transpose_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_atrium(transpose_folders[folder],ext)

# Train/weight folder
# weight_folders = {}
for folder in weight_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_atrium(weight_folders[folder],ext)

# Train/segmented_ids folder
# id_folders = {}
for folder in id_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_atrium(id_folders[folder],ext)

### Remove files from ventricle

In [ ]:
# Setting the directories
import os
cwd = os.getcwd() + '/datasets-24-aug/OCTData/Ventricle/'

print(cwd)
directories = sorted(os.listdir(path=cwd))
directories = directories[1:]

# Standard folder
folders = {}
for i in directories:
    name = i+'_folder'
    folders[name] = cwd + i+'/'
print('Name: {}\nDirectory: {}\n'.format('whole_raw_image_folder', folders['whole_raw_image_folder']))

# Train folder
train_folders = {}
for i in directories:
    name = i
    train_folders[name] = cwd + i+'/Train/'

# Transpose folder
transpose_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/transpose/'):
        transpose_folders[name] = cwd + i+'/transpose/'

# Train/weight folder
weight_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/weights/'):
        weight_folders[name] = cwd + i+'/Train/weights/'
    
# Train/segmented_ids folder
id_folders = {}
for i in directories:
    name = i
    if os.path.exists(cwd + i+'/Train/segmented_ids/'):
        id_folders[name] = cwd + i+'/Train/segmented_ids/'
    

In [ ]:
# Standard folder - .tif
for folder in folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_ventricle(folders[folder],ext)

# Train folder - .tif
# train_folders = {}
for folder in train_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_ventricle(train_folders[folder],ext)

# Transpose folder
# transpose_folders = {}
for folder in transpose_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_ventricle(transpose_folders[folder],ext)

# Train/weight folder
# weight_folders = {}
for folder in weight_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_ventricle(weight_folders[folder],ext)

# Train/segmented_ids folder
# id_folders = {}
for folder in id_folders: 
    for ext in ['.tif', '.jpg', '.JPG', '.png', '.npy']:
        remove_files_ventricle(id_folders[folder],ext)